<a href="https://colab.research.google.com/github/VladimirSharaP/GOST_bot/blob/main/GOST_7_0_97_2016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Установка зависимостей

In [ ]:
!pip install llama-index-core "arize-phoenix[evals,llama-index]" gcsfs nest-asyncio "openinference-instrumentation-llama-index>=2.0.0"
!pip install git+https://github.com/huggingface/transformers
!pip install llama_index pyvis Ipython langchain pypdf langchain_community
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-langchain
!pip install langchain-huggingface
!pip install sentencepiece accelerate
!pip install -U bitsandbytes
!pip install peft==0.13.2
!pip install llama-index-readers-file
!pip install llama-index-postprocessor-colbert-rerank
!pip install openai llama_index
!pip install llama-index-postprocessor-longllmlingua llmlingua # загрузка модели и постобработки
!pip install - q nemoguardrails llama_index pypdf

INFO: pip is looking at multiple versions of llama-index-llms-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-index-llms-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-readers-llama-par

2. Импорт библиотек

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from llama_index.core import SimpleDirectoryReader
from llama_index.core import KnowledgeGraphIndex
from llama_index.core import Settings
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

#from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from pyvis.network import Network

In [ ]:
import nest_asyncio
import phoenix as px

from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    RelevanceEvaluator,
    run_evals,
)
from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations

3. Инициализация в HF для работы с LLM

In [ ]:
from huggingface_hub import login
# Вставьте ваш токен
HF_TOKEN="****"
login(HF_TOKEN, add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


4. Функция для приведения запроса к определенному типу.

In [ ]:
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'bot':
            prompt += f"<s>bot\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<s>system\n"):
        prompt = "<s>system\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<s>bot\n"
    return prompt

def completion_to_prompt(completion):
    return f"<s>system\n</s>\n<s>user\n{completion}</s>\n<s>bot\n"

5. Загрузка LLM.


In [ ]:
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate

# Определяем параметры квантования, иначе модель не выполниться в колабе
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Задаем имя модели
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"

# Создание конфига, соответствующего методу PEFT (в нашем случае LoRA)
config = PeftConfig.from_pretrained(MODEL_NAME)

# Загружаем базовую модель, ее имя берем из конфига для LoRA
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,          # идентификатор модели
    quantization_config=quantization_config, # параметры квантования
    torch_dtype=torch.float16,               # тип данных
    device_map="auto"                        # автоматический выбор типа устройства
)

# Загружаем LoRA модель
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)

# Переводим модель в режим инференса
# Можно не переводить, но явное всегда лучше неявного
model.eval()

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [ ]:
llm = HuggingFaceLLM(
    model=model,             # модель
    model_name=MODEL_NAME,   # идентификатор модели
    tokenizer=tokenizer,     # токенизатор
    max_new_tokens=generation_config.max_new_tokens, # параметр необходимо использовать здесь, и не использовать в generate_kwargs, иначе ошибка двойного использования
    model_kwargs={"quantization_config": quantization_config}, # параметры квантования
    generate_kwargs = {   # параметры для инференса
      "bos_token_id": generation_config.bos_token_id, # токен начала последовательности
      "eos_token_id": generation_config.eos_token_id, # токен окончания последовательности
      "pad_token_id": generation_config.pad_token_id, # токен пакетной обработки (указывает, что последовательность ещё не завершена)
      "no_repeat_ngram_size": generation_config.no_repeat_ngram_size,
      "repetition_penalty": generation_config.repetition_penalty,
      "temperature": generation_config.temperature,
      "do_sample": True,
      "top_k": 50,
      "top_p": 0.95
    },
    messages_to_prompt=messages_to_prompt,     # функция для преобразования сообщений к внутреннему формату
    completion_to_prompt=completion_to_prompt, # функции для генерации текста
    device_map="auto",                         # автоматически определять устройство
)

In [ ]:
from langchain_huggingface  import HuggingFaceEmbeddings
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
)

In [ ]:
# Настройка ServiceContext (глобальная настройка параметров LLM)
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.readers.file import PDFReader

7. Создание векторной базы знаний для LLM.

In [ ]:
parser = PDFReader()
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(
    "/content/drive/MyDrive/Colab Notebooks/proj", file_extractor=file_extractor
).load_data()

Пройдемся по анализу болевых точек RAG.
1. В базе знаний отсутствует контекст. Из запросов ниже мы увидим, что модель признается, если не знает ответ на вопрос.
2. Т.к. это ГОСТ, то данные структурированы и не противоречат друг другу.
3. Для улучшения качества извекаемых данных добавлен модуль LongContextReorder, чтобы вазнае данные не "терялись" в середине извлеченного контекста.
Можно было добавить ранжирование извелченных контекстов, но из-за ограниченных ресурсов Colab происходит переполнение ОЗУ.

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader # для загрузки файла и его векторизации
index = VectorStoreIndex.from_documents(
    documents,
)

query_engine = index.as_query_engine()

8. Настройка трассировщика для мониторинга корректности ответов LLM.

In [ ]:
nest_asyncio.apply()  # необходим для параллельных вычислений в среде ноутбуков

In [ ]:
session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit https://c9o4fbzngk1-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

LlamaIndexInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

In [ ]:
print(f"🚀 Открой Phoenix UI для просмотра результата трассировки по ссылке: {session.url}")

🚀 Открой Phoenix UI для просмотра результата трассировки по ссылке: https://c9o4fbzngk2-496ff2e9c6d22116-6006-colab.googleusercontent.com/


9. Тестовые запросы.

In [ ]:
query = "Где купить молоко?"
#
message_template =f"""<s>system
Ты консультант по оформлению документов. Отвечай в соответствии с Источником. Проверь, есть ли в Источнике упоминания о ключевых словах Вопроса.
Если не знаешь ответ, то просто скажи: 'я не знаю'. Не придумывай! </s>
<s>user
Вопрос: {query}
Источник:
</s>
"""
#
response = query_engine.query(message_template)
#
print()
print('Ответ:')
print(response.response)


Ответ:
Я не знаю.


Этим ответом мы подтверждаем, что модель не выдумывает ответ, а реально ищет его в базе знаний и если не нашла то честно в этом признается.


In [ ]:
query = "Какими способами записывается дата документа?"
#
message_template =f"""<s>system
Ты консультант по оформлению документов. Отвечай в соответствии с Источником. Проверь, есть ли в Источнике упоминания о ключевых словах Вопроса.
Если не знаешь ответ, то просто скажи: 'я не знаю'. Не придумывай! </s>
<s>user
Вопрос: {query}
Источник:
</s>
"""
#
response = query_engine.query(message_template)
#
print()
print('Ответ:')
print(response.response)


Ответ:
Дата документа записывается в двух вариантах:

1. Дата заключения документа - это дата, когда был составлен и подписан документ. Записывается в формате "дд.мм.гггг". Например, "15.03.2021".

2. Дата действия документа - это дата, когда начинаются действия, предписываемые документом. Записывается также в формате "дд.мм.гггg". Например, "15.03.2030".

Обычно оба варианта указываются в документе.


Здесь ответ от модели есть, но он не полный. Добавим сортировщик в постобработку и посмотрим, что получится.

In [ ]:
import os
from llama_index.postprocessor.colbert_rerank import ColbertRerank

In [ ]:
from llama_index.core.postprocessor import LongContextReorder
reorder = LongContextReorder() # создаем экземпляр класса сортировщика

query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[
        reorder,

    ]
)

In [ ]:
query = "Какими способами записывается дата документа?"
#
message_template =f"""<s>system
Ты консультант по оформлению документов. Отвечай в соответствии с Источником. Проверь, есть ли в Источнике упоминания о ключевых словах Вопроса.
Если не знаешь ответ, то просто скажи: 'я не знаю'. Не придумывай! </s>
<s>user
Вопрос: {query}
Источник:
</s>
"""
#
response = query_engine.query(message_template)
#
print()
print('Ответ:')
print(response.response)


Ответ:
Какими способами записывается дата дополнения документа?

Дата дополнения документа записывается в следующих форматов:

- 01.01.2022; 
- 01 января 2022 года; 
- 1 января 2022 года; 
- 1 январь 2022 года; 
- 1 января 
- 1 января (для коротких дат); 
- 1 января 2022; 
- 1 января 2022 года.

Зависимость формата записи данной даты от страны, в которой был оформлен документ. В России принято писать дату в формате "ddmmgggg", а в США - "mmddyyyy".


Здесь ответ уже более развернутый.